In [77]:
# coding:utf-8
'''
本脚本要完成的功能是对IEMOCAP和ravdess两个数据集数组化，
每一个wav用一个不定长的数组表示，并配上标签
对于IEMOCAP数据，总共11个标签，label数组为['Neutral', 'Frustration', 'Other', 
'Disgust', 'Surprise', 'Excited', 'Anger', 'Fear', 'Sadness', 'Happiness', 'Overlap']
对于RAVDESS,总共8个标签，为['netural','calm','happy','sad','angry','fearful','disgust','surprised']
'''

import os
from os import listdir
from os.path import join
import scipy
from scipy.io import wavfile as wav
import matplotlib.pyplot as plt
from scipy import signal
from sklearn import preprocessing
import numpy as np
import time
import random
from tqdm import tqdm
import struct
from PIL import Image

IEMOCAP_EMOTIONS={'Neutral state':0, 'Frustration':1, 'Other':2,\
'Disgust':3, 'Surprise':4, 'Excited':5, 'Anger':6, 'Fear':7, 'Sadness':8, 'Happiness':9, 'Overlap':10}
IEMOCAP_EMOTIONS_LIST=['Neutral state', 'Frustration', 'Other',\
'Disgust', 'Surprise', 'Excited', 'Anger', 'Fear', 'Sadness', 'Happiness', 'Overlap']

RAVDESS_EMOTIONS=['netural','calm','happy','sad','angry','fearful','disgust','surprised']

IEMOCAP_DIR='D:\\David Zhou\\Stressful Detection Project\\data\\Emotion Dataset\\IEMOCAP_full_release'
RAVDESS_DIR='D:\\David Zhou\\Stressful Detection Project\\data\\Emotion Dataset\\RAVDESS-SpeechAO_AllActors'

RAVDESS_PREPROCESSED_DIR = 'D:\\David Zhou\\Stressful Detection Project\\data\\preprocessed data\\RAVDESS'
IEMOCAP_PREPROCESSED_DIR = 'D:\\David Zhou\\Stressful Detection Project\\data\\preprocessed data\\IEMOCAP'

def findAllFilePaths(root,suffix='.wav'):
    # give a directory and return all file's absolute paths ending with {suffix}
    wavFilePaths = list()
    queue = list()
    queue.append(root)
    while (len(queue)):
        currentDir = queue.pop(0)
        if os.path.isdir(currentDir):
            queue.extend([join(currentDir,d) for d in listdir(currentDir) if os.path.isdir(join(currentDir,d))])
            wavFilePaths.extend([join(currentDir,w) for w in listdir(currentDir) if w.endswith(suffix)])
    return wavFilePaths

def readAllWaves(paths):
    '''
    give wav absolute paths and return 
    a list of numpy array which is scaled
    '''
    scaled_wavs = list()
    for path in paths:
        sample_rate,wav_data = wav.read(path)
        if(len(wav_data.shape)>1):
            # 处理多通道情形
            wav_data = wav_data.T[0]
        scaled_wav_data = preprocessing.scale(wav_data)
        scaled_wavs.append(scaled_wav_data)
    return scaled_wavs

def readRavdessLabels(paths):
    '''
    give a list of file paths
    return a list of integer labels 
    starting from one
    corresponding to each path
    '''
    return [int(path.split('\\')[-1].split('-')[2]) for path in paths]

def readIemocapLabels(cat_paths):
    '''give label paths and return a dict,key is wavfile name, 
    value is integer label, starting from zero'''
    label_dict=dict()
    for filename in cat_paths:
        file = open(filename,'r')
        records = file.readlines()
        for r in records:
            key = r.split(' ')[0]
            er = r.split(':')[1:]  #去掉第一个
            er = [e.split(';')[0] for e in er]
            vals = [IEMOCAP_EMOTIONS[e] for e in er]
            if not key in label_dict:
                label_dict[key]=vals
            else:
                label_dict[key].extend(vals)
        file.close()
    return label_dict


In [75]:
# 处理RAVDESS数据
ravdess_paths = findAllFilePaths(RAVDESS_DIR)
# print('processing data')
# ravdess_data = readAllWaves(ravdess_paths)
ravdess_labels = readRavdessLabels(ravdess_paths)

# 给每个wav array贴上标签
# print('tagging')
# labeled_ravdess_data = list()
# for i in range(len(ravdess_labels)):
# 	labeled_ravdess_data.append((ravdess_data[i],ravdess_labels[i]))
# print('saving')
# np.save(os.path.join(RAVDESS_PREPROCESSED_DIR,'RAVDESS_SERIES.npy'),labeled_ravdess_data)

In [78]:
# 对音频进行分类
# 新建八个分类目录
for name in IEMOCAP_EMOTIONS.keys():
    try:
        os.mkdir(os.path.join(IEMOCAP_DIR,name))
    except:
        pass

In [79]:
# 处理IEMOCAP
iemocap_paths = findAllFilePaths(IEMOCAP_DIR)
cat_paths = findAllFilePaths(IEMOCAP_DIR,suffix='cat.txt')
# print('processing data')
# iemocap_data = readAllWaves(iemocap_paths)
iemocap_label_dict = readIemocapLabels(cat_paths)

In [81]:
iemocap_label_dict['Ses01F_impro01_F010']

[1, 1, 4, 3, 6, 6]

In [21]:
# 给每个data贴标签
labeled_iemocap_data = list()
print('tagging')
for i in range(len(iemocap_paths)):
    key = iemocap_paths[i].split('\\')[-1][0:-4]
    label = iemocap_label_dict[key]
    labeled_iemocap_data.append((iemocap_data[i],label))
print('saving')
np.save(os.path.join(IEMOCAP_PREPROCESSED_DIR,'IEMOCAP_SERIES.npy'),labeled_iemocap_data)

processing data


d:\python35\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int16 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


tagging
saving


In [24]:
# 读取并分类
IEMOCAP_EMOTIONS_LIST=['Neutral', 'Frustration', 'Other',\
'Disgust', 'Surprise', 'Excited', 'Anger', 'Fear', 'Sadness', 'Happiness', 'Overlap']
for path in iemocap_paths:
    tmp = open(path,'rb').read()
    filename = path.split('\\')[-1][0:-4] # say Ses01F_impro01_F000
    label_index = iemocap_label_dict[filename]
    label_str = IEMOCAP_EMOTIONS_LIST[label_index]
    out = open(os.path.join(IEMOCAP_DIR,os.path.join(label_str,filename)+'.wav'),'wb')
    out.write(tmp)
    out.close()

In [41]:
iemocap_label_dict['Ses01F_impro01_M011']

1

In [48]:
min = 1e10
for i in range(len(ravdess_data)):
    if ravdess_data[i].shape[0]<min:
        min = ravdess_data[i].shape[0]
min

140941